In [1]:
import urllib
from IPython.display import Image, display, clear_output
from collections import Counter

#import matplotlib.pyplot as plt
#import seaborn as sns 
#%matplotlib inline

import json
from sklearn.metrics import classification_report, confusion_matrix
from keras.optimizers import SGD, RMSprop
from keras.constraints import maxnorm
#sns.set_style('whitegrid')

C:\Users\pankaj.prabhakar\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [4]:
import os
import h5py
import numpy as np
import pandas as pd

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2, l1
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint, History

In [29]:
# path to the model weights file
#location = 'C:\\Users\\pankaj.prabhakar\\Desktop\\defect_detection\\injury'
location = 'C:\\Users\\pankaj.prabhakar\\Desktop\\mobile\\mobile-20181129T162351Z-001\\mobile'
#location = 'C:\\Users\\pankaj.prabhakar\\Desktop\\Total_Partial\\data'
top_model_weights_path=location+'\\top_model_weights.h5' # will be saved into when we create our model
# model_path = location + '/initial_data2_model.h5'
fine_tuned_model_path = location+'\\scratch-vs-nonscratch.h5'

# dimensions of our images
img_width, img_height = 100, 100

#train_data_dir = location+'\\train'
train_data_dir = location+'\\train'
validation_data_dir = location+'\\test'

train_samples = [len(os.listdir(train_data_dir+'\\'+i)) for i in sorted(os.listdir(train_data_dir))]
nb_train_samples = sum(train_samples)
validation_samples = [len(os.listdir(validation_data_dir+'\\'+i)) for i in sorted(os.listdir(validation_data_dir))]
nb_validation_samples = sum(validation_samples)

In [18]:
nb_validation_samples

7

In [19]:
fine_tuned_model_path

'C:\\Users\\pankaj.prabhakar\\Desktop\\mobile\\mobile-20181129T162351Z-001\\mobile\\scratch-vs-nonscratch.h5'

In [20]:
#weights_path='C:\\Users\\pankaj.prabhakar\\Desktop\\defect_detection\\vgg16_weights.h5'

In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
#**import tkinter as tk
#from tkinter import filedialog
import os.path
import numpy as np

#fix random seed for reproducibility
seed = 2017
np.random.seed(seed)

img_width, img_height = 100, 100

epochs = 50
batch_size = 5
lrate = 0.0001
decay = 1e-6
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
rmsprop = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-8, decay=0.0)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

#this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#this is the augmentation configuration we will use for testing:
#only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

checkpoint = ModelCheckpoint(fine_tuned_model_path, monitor='val_acc', 
                                 verbose=1, save_best_only=True, 
                                 save_weights_only=False, mode='auto')
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=1,
    callbacks=[checkpoint])

#with open(location+'\\ft_history.txt', 'wb') as f:
#        json.dump(fit.history, f)

Found 15 images belonging to 2 classes.
Found 7 images belonging to 2 classes.
Epoch 1/50
3/3 [==============================] - 2s 637ms/step - loss: 0.7301 - acc: 0.4667 - val_loss: 0.7138 - val_acc: 0.6000

Epoch 00001: val_acc improved from -inf to 0.60000, saving model to C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\scratch-vs-nonscratch.h5
Epoch 2/50
3/3 [==============================] - 1s 176ms/step - loss: 0.7950 - acc: 0.3333 - val_loss: 0.6687 - val_acc: 1.0000

Epoch 00002: val_acc improved from 0.60000 to 1.00000, saving model to C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\scratch-vs-nonscratch.h5
Epoch 3/50
3/3 [==============================] - 1s 172ms/step - loss: 0.6764 - acc: 0.6667 - val_loss: 0.7165 - val_acc: 0.6000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/50
3/3 [==============================] - 1s 184ms/step - loss: 0.6778 - acc: 0.5333 - val_loss: 0.6521 - val_acc: 1.0000

Epoch 

3/3 [==============================] - 1s 182ms/step - loss: 0.3856 - acc: 0.8667 - val_loss: 0.8200 - val_acc: 0.5000

Epoch 00044: val_acc did not improve from 1.00000
Epoch 45/50
3/3 [==============================] - 1s 194ms/step - loss: 0.3176 - acc: 0.8667 - val_loss: 0.5849 - val_acc: 0.8000

Epoch 00045: val_acc did not improve from 1.00000
Epoch 46/50
3/3 [==============================] - 1s 198ms/step - loss: 0.3493 - acc: 0.8000 - val_loss: 0.6410 - val_acc: 0.5000

Epoch 00046: val_acc did not improve from 1.00000
Epoch 47/50
3/3 [==============================] - 1s 199ms/step - loss: 0.3285 - acc: 0.8667 - val_loss: 0.7270 - val_acc: 0.6000

Epoch 00047: val_acc did not improve from 1.00000
Epoch 48/50
3/3 [==============================] - 1s 193ms/step - loss: 0.4112 - acc: 0.7333 - val_loss: 0.3540 - val_acc: 1.0000

Epoch 00048: val_acc did not improve from 1.00000
Epoch 49/50
3/3 [==============================] - 1s 206ms/step - loss: 0.3902 - acc: 0.8667 - val_lo

In [24]:
def prepare_img_256(img_path):
	img = load_img(img_path, target_size=(100, 100))
	x = img_to_array(img)
	x = x.reshape((1,) + x.shape)/255
	return x

In [40]:
def get_damage_non_damage(img_256, model):
    print ("Determining scratch...")
    pred = model.predict(img_256)
    if pred[0][0] <=.5:
        return 'No'
    else:
        return 'Yes'

In [41]:
damage_detection = load_model('scratch-vs-nonscratch.h5')

In [42]:
data_path = r'C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test'

In [43]:
for path, subdirs, files in os.walk(data_path):
    for name in files:
        image_path = os.path.join(path, name)
        print(image_path)
        img_256 = prepare_img_256(image_path)
        dnd = get_damage_non_damage(img_256, damage_detection)
        print(dnd)

C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test\nonscratch\index3.jpeg
Determining scratch...
Yes
C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test\nonscratch\iPad-Air.jpg
Determining scratch...
No
C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test\nonscratch\simp.png
Determining scratch...
Yes
C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test\scratch\Second.jpg
Determining scratch...
Yes
C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test\scratch\Test1.PNG
Determining scratch...
Yes
C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test\scratch\Test2.PNG
Determining scratch...
Yes
C:\Users\pankaj.prabhakar\Desktop\mobile\mobile-20181129T162351Z-001\mobile\test\scratch\Third.PNG
Determining scratch...
No
